In [1]:
import sqlalchemy as sa
import urllib
import pandas as pd
import pyodbc
import time
import glob
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
from  tqdm import tqdm
pd.set_option('display.max_columns', 41)

connection_string = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'SERVER=118.69.201.34;'
    'Database=FLC_SHOP_SALES_DATAWAREHOUSE;'
    'UID=ecom_user;'
    'PWD=Ec0m@12345;'
    'Trusted_Connection=no;'
)

connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(str(connection_string))}"
engine = sa.create_engine(connection_uri, fast_executemany=True)

In [2]:
def getEngine(engine_name):
    connection = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(engine_name)}"
    engine = sa.create_engine(connection, fast_executemany=True)
    return engine
    
### Connect to DataMart Server
mart_server = '118.69.201.34'
mart_database = 'FLC_SHOP_SALES_DATAWAREHOUSE'
mart_username = 'ecom_user'
mart_password = 'Ec0m@12345'
SaleMart = ('DRIVER={ODBC Driver 17 for SQL Server}'
            ';SERVER=' + mart_server +
            ';DATABASE=' + mart_database +
            ';UID=' + mart_username +
            ';PWD=' + mart_password)

### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'
        
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)

azr_engine = sa.create_engine(connection_url)

In [11]:
# conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+mart_server+';DATABASE='+mart_database+';UID='+mart_username+';PWD='+ mart_password)
# cursor = conn.cursor()
# # Câu lệnh SQL để tạo bảng
# create_table_1 = '''
# CREATE TABLE FOLLOW_TRANSACTION_FLC (
#     ID INT IDENTITY(1,1) PRIMARY KEY,
#     DATE_KEY INT,
#     COUNT_LINE INT,
#     INSERT_BY NVARCHAR(20),
#     INSERT_DATE DATETIME DEFAULT GETDATE())
# '''

# create_table_2 = '''
# CREATE TABLE FOLLOW_TRANSACTION_ICT (
#     ID INT IDENTITY(1,1) PRIMARY KEY,
#     DATE_KEY INT,
#     COUNT_LINE INT,
#     INSERT_BY NVARCHAR(20),
#     INSERT_DATE DATETIME DEFAULT GETDATE())
# '''


# # Thực thi câu lệnh tạo bảng
# cursor.execute(create_table_1)
# cursor.execute(create_table_2)

# # Lưu các thay đổi vào cơ sở dữ liệu
# conn.commit()

# # Đóng kết nối và con trỏ
# cursor.close()
# conn.close()

In [3]:
sales_trans_flc = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'trucvhc' AS INSERT_BY
FROM dbo.VF_CID_SALES_TRANSACTION_FLC
GROUP BY DATE_KEY
ORDER BY DATE_KEY"""
trans_flc = pd.read_sql_query(sales_trans_flc,azr_engine)
sales_trans_ict = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'trucvhc' AS INSERT_BY
FROM dbo.VF_CID_SALES_TRANSACTION_ICT
GROUP BY DATE_KEY
ORDER BY DATE_KEY"""
trans_ict = pd.read_sql_query(sales_trans_ict,azr_engine)
shop_lc = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'trucvhc' AS INSERT_BY
FROM dbo.VF_CID_SALES_TRANSACTION_ICT
GROUP BY DATE_KEY
ORDER BY DATE_KEY"""

E:\Anaconda\lib\site-packages\pandas\io\sql.py:1405: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]User does not have permission to perform this action. (6004) (SQLExecDirectW)')
  return self.connectable.execution_options().execute(*args, **kwargs)


In [ ]:
# # Thông tin kết nối cơ sở dữ liệu PostgreSQL
# username = "cid"
# password = "cid123456"
# host = "localhost"
# port = "5432"
# database = "FLC"
# # Tạo đối tượng engine
# engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
# table_name = "sale_transaction_flc"

In [4]:
# csv_file = r"D:\CHECK_DATA\CUSTOMER_ALL_20230622.csv"
# df_customer_all = pd.read_csv(csv_file)
# df_customer_all

In [ ]:
# n = 50000
# table_name = "CID_CUSTOMER_FLC"
# error = []
# df_parts = [df_customer_all[i:i+n] for i in range(0, df_customer_all.shape[0], n)]
# for i in tqdm(range(len(df_parts))):
#     try:
#         df_parts[i].to_sql(table_name, engine, if_exists='append', index=False)
#     except:
#         error.append(df_parts[i])
#     time.sleep(1)

In [ ]:
# error_rows = []
# for _, row in df_result.iterrows():
#     try:
#         row_df = pd.DataFrame([row])
#         row_df.to_sql(table_name, engine, if_exists='append', index=False)
#     except Exception as e:
#         print(f"Error inserting row: {row}\nError message: {str(e)}")
#         error_rows.append(row)

In [ ]:
# error_df = pd.DataFrame(error_rows)

In [5]:
# Lấy thời gian hiện tại
current_time = datetime.now()
formatted_time = current_time.strftime("%Y%m%d")
trans_flc['DATE_KEY'] = trans_flc['DATE_KEY'].astype(str)
trans_ict['DATE_KEY'] = trans_ict['DATE_KEY'].astype(str)
trans_flc = trans_flc.loc[trans_flc['DATE_KEY'] < formatted_time]
trans_ict = trans_ict.loc[trans_ict['DATE_KEY'] < formatted_time]
trans_flc

,DATE_KEY,COUNT_LINE,INSERT_BY
0,20190101,12959,trucvhc
1,20190102,15480,trucvhc
2,20190103,15803,trucvhc
3,20190104,14740,trucvhc
4,20190105,14325,trucvhc
...,...,...,...
1633,20230622,499285,trucvhc
1634,20230623,540404,trucvhc
1635,20230624,500329,trucvhc
1636,20230625,536636,trucvhc


In [6]:
trans_flc.to_sql("FOLLOW_TRANSACTION_FLC", engine, schema="dbo", if_exists="append", index=False)
trans_ict.to_sql("FOLLOW_TRANSACTION_ICT", engine, schema="dbo", if_exists="append", index=False)

-1

In [8]:
csv_file = r"D:\CHECK_DATA\TNBQ.csv"
chunk_size = 100000  # Đọc 1 triệu dòng mỗi lần
df_chunks = pd.read_csv(csv_file, chunksize=chunk_size) 
table_name = "CID_LOYALTY_FS"
for chunk in df_chunks:
    chunk.to_sql(table_name, engine, if_exists='append', index=False)

In [6]:
df_check = pd.read_excel(r"D:\CHIPHISHOP\TNBQ.xlsx")

In [8]:
df_check.to_sql('CID_ECONOMICCENSUS_2021', engine, if_exists='append', index=False)

-1

In [78]:
check_point_use = f"""SELECT *
FROM dbo.CID_LOYALTY_FS
WHERE TransactionPointType IN ('205','202')
AND OrderCode IS NULL
ORDER BY TransactionDate"""
trans_point_use = pd.read_sql_query(check_point_use,engine)

In [79]:
trans_point_use

,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
0,2E517A09420928E49AD5CEB5371C6B55,3000,3000.0,0.0,205,2022-10-15 06:54:25.6033333,None
1,17380A24D2B0E50D1838E8B0382E534B,1500,1500.0,0.0,205,2022-10-15 08:00:10.5700000,None
2,C13A6BA8A8E8C7CF97CC828F75B51196,4500,4500.0,0.0,205,2022-10-15 08:31:22.9200000,None
3,1C58092FED20EAB06A599381A6C87F94,1500,1500.0,0.0,205,2022-10-15 08:47:17.1133333,None
4,3819C6E0DD689AE7012440A882B67AAA,1500,1500.0,0.0,205,2022-10-15 08:49:24.1266667,None
...,...,...,...,...,...,...,...
5909,92AFC68BF5B2D65DA6C82B28CD7A4890,1500,1500.0,15000.0,205,2023-06-26 06:34:13.1636854,None
5910,AA04F0ED601F9FAE30A9E5858D6930FC,1500,1500.0,15000.0,205,2023-06-26 06:34:19.9434003,None
5911,7109A60EE66AE05ABF9D0A51AB751E1F,3000,3000.0,30000.0,205,2023-06-26 06:34:49.0155466,None
5912,8B374B909AEF6AA98893C3D0B41D7F0F,1500,1500.0,15000.0,205,2023-06-26 06:35:11.1739833,None


,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
0,2E517A09420928E49AD5CEB5371C6B55,3000,3000.0,0.0,205,2022-10-15 06:54:25.6033333,None
1,17380A24D2B0E50D1838E8B0382E534B,1500,1500.0,0.0,205,2022-10-15 08:00:10.5700000,None
2,C13A6BA8A8E8C7CF97CC828F75B51196,4500,4500.0,0.0,205,2022-10-15 08:31:22.9200000,None
3,1C58092FED20EAB06A599381A6C87F94,1500,1500.0,0.0,205,2022-10-15 08:47:17.1133333,None
4,3819C6E0DD689AE7012440A882B67AAA,1500,1500.0,0.0,205,2022-10-15 08:49:24.1266667,None
...,...,...,...,...,...,...,...
5909,92AFC68BF5B2D65DA6C82B28CD7A4890,1500,1500.0,15000.0,205,2023-06-26 06:34:13.1636854,None
5910,AA04F0ED601F9FAE30A9E5858D6930FC,1500,1500.0,15000.0,205,2023-06-26 06:34:19.9434003,None
5911,7109A60EE66AE05ABF9D0A51AB751E1F,3000,3000.0,30000.0,205,2023-06-26 06:34:49.0155466,None
5912,8B374B909AEF6AA98893C3D0B41D7F0F,1500,1500.0,15000.0,205,2023-06-26 06:35:11.1739833,None


In [80]:
merged_df = pd.merge(trans_point_use, df_check, on=['phonenumber', 'TransactionDate','transactionPoint','ActualPoint','TransactionAmount','TransactionPointType'], how='inner')
filtered_df = merged_df[merged_df['OrderCode_y'].notna()]
filtered_df = filtered_df.drop(filtered_df.columns[[6]], axis=1).drop_duplicates()
filtered_df.rename(columns={filtered_df.columns[6]: 'OrderCode'}, inplace = True)
filtered_df

,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode


,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode


In [41]:
filtered_df

,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
5257,8ED87DD2626D46D5182AFCEB16D644FE,4500,4500.0,64500.0,202,2023-06-22 16:25:55.8565003,80353404731687426076095
5369,82B1FD789A5378FA345F1B9DFAF1E0A0,6000,6000.0,65000.0,205,2023-06-23 09:05:13.0623671,80381423761687485968014
5399,4252DBC1B213599B369FD2720E2E7A8B,15000,15000.0,210000.0,202,2023-06-23 10:56:15.2812495,80052016321687492753379
5638,7ADAC81D26F4EE37C535FCDD4EAEC0AE,3000,3000.0,52000.0,202,2023-06-24 16:35:17.2924116,80363427831687599743557
5721,31C94C51FA370A614A5717BFFBAB2E3F,4500,4500.0,56000.0,202,2023-06-25 07:21:39.7388327,80470089691687652604452
5855,A0287817F3D2F0395D61A3D97C8080C8,3000,3000.0,30000.0,205,2023-06-25 17:44:28.7332810,80414054321687689872257


,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
5257,8ED87DD2626D46D5182AFCEB16D644FE,4500,4500.0,64500.0,202,2023-06-22 16:25:55.8565003,80353404731687426076095
5369,82B1FD789A5378FA345F1B9DFAF1E0A0,6000,6000.0,65000.0,205,2023-06-23 09:05:13.0623671,80381423761687485968014
5399,4252DBC1B213599B369FD2720E2E7A8B,15000,15000.0,210000.0,202,2023-06-23 10:56:15.2812495,80052016321687492753379
5638,7ADAC81D26F4EE37C535FCDD4EAEC0AE,3000,3000.0,52000.0,202,2023-06-24 16:35:17.2924116,80363427831687599743557
5721,31C94C51FA370A614A5717BFFBAB2E3F,4500,4500.0,56000.0,202,2023-06-25 07:21:39.7388327,80470089691687652604452
5855,A0287817F3D2F0395D61A3D97C8080C8,3000,3000.0,30000.0,205,2023-06-25 17:44:28.7332810,80414054321687689872257


In [65]:
phone_arr = filtered_df['phonenumber'].values
phone_list = phone_arr.tolist()
date_arr = filtered_df['TransactionDate'].values
date_list = date_arr.tolist()

In [66]:
# Chuyển đổi danh sách các chuỗi thành một chuỗi đúng để sử dụng trong truy vấn SQL
phone = "'" + "','".join(phone_list) + "'"
date = "'" + "','".join(date_list) + "'"

In [76]:
replace_point_use = f"""SELECT *
FROM dbo.CID_LOYALTY_FS
WHERE phonenumber IN ({phone})
AND TransactionPointType IN ('202','205')
AND TransactionDate IN ({date})"""
rs_replace_point_use = pd.read_sql_query(replace_point_use,engine)

In [77]:
rs_replace_point_use

,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
0,4252DBC1B213599B369FD2720E2E7A8B,15000,15000.0,210000.0,202,2023-06-23 10:56:15.2812495,80052016321687492753379
1,31C94C51FA370A614A5717BFFBAB2E3F,4500,4500.0,56000.0,202,2023-06-25 07:21:39.7388327,80470089691687652604452
2,82B1FD789A5378FA345F1B9DFAF1E0A0,6000,6000.0,65000.0,205,2023-06-23 09:05:13.0623671,80381423761687485968014
3,7ADAC81D26F4EE37C535FCDD4EAEC0AE,3000,3000.0,52000.0,202,2023-06-24 16:35:17.2924116,80363427831687599743557
4,8ED87DD2626D46D5182AFCEB16D644FE,4500,4500.0,64500.0,202,2023-06-22 16:25:55.8565003,80353404731687426076095
5,A0287817F3D2F0395D61A3D97C8080C8,3000,3000.0,30000.0,205,2023-06-25 17:44:28.7332810,80414054321687689872257


,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode
0,4252DBC1B213599B369FD2720E2E7A8B,15000,15000.0,210000.0,202,2023-06-23 10:56:15.2812495,80052016321687492753379
1,31C94C51FA370A614A5717BFFBAB2E3F,4500,4500.0,56000.0,202,2023-06-25 07:21:39.7388327,80470089691687652604452
2,82B1FD789A5378FA345F1B9DFAF1E0A0,6000,6000.0,65000.0,205,2023-06-23 09:05:13.0623671,80381423761687485968014
3,7ADAC81D26F4EE37C535FCDD4EAEC0AE,3000,3000.0,52000.0,202,2023-06-24 16:35:17.2924116,80363427831687599743557
4,8ED87DD2626D46D5182AFCEB16D644FE,4500,4500.0,64500.0,202,2023-06-22 16:25:55.8565003,80353404731687426076095
5,A0287817F3D2F0395D61A3D97C8080C8,3000,3000.0,30000.0,205,2023-06-25 17:44:28.7332810,80414054321687689872257
